# Pset 4

In [76]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [77]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [114]:
demand = CSV.read("data/demand.csv", DataFrame)[:, 2:501]
facilities = CSV.read("data/facilities.csv", DataFrame)
production = CSV.read("data/production.csv", DataFrame)
transportation = CSV.read("data/transportation.csv", DataFrame)
zones = CSV.read("data/zones.csv", DataFrame)


num_facilities = size(facilities, 1)
num_customers = size(demand, 1)
num_scenarios = size(demand, 2)

c_j = Vector(facilities.cost)
C_j = Vector(facilities.capacity)
t_ij = Matrix(transportation)
u_i = Vector(zones.unmet)
D_is = Matrix(demand)
q_js = Matrix(production)

p = 1/num_scenarios * ones(num_scenarios)

println(facilities.cost)
println(size(facilities.cost))
println(num_facilities)
println(num_customers)
println(num_scenarios)
println(p)
print(size(p))

[65.34564412755189, 46.975898434445725, 201.45602729941479, 25.250772992020554, 196.78632375277724, 50.144830986003555, 30.06603532913156, 81.5939614380697, 83.62967824349536, 71.52260340118428, 45.520944342391395, 93.62118168471744, 7.859779072506969, 49.92747178980714, 13.148411972669551, 60.16848463801613, 131.18357389529552, 118.77846387222779, 50.86192989336156, 84.47463261765012]
(20,)
20
50
500
[0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,

In [ ]:
# CSV.File("unmet_demand_cost.csv",header=0) 
demand = CSV.File("data/demand.csv", header=0)|> Tables.matrix;
facilities = CSV.File("data/facilities.csv", header=0)|> Tables.matrix;
production = CSV.File("data/production.csv", header=0)|> Tables.matrix;
transportation = CSV.File("data/transportation.csv", header=0)|> Tables.matrix;
zones = CSV.File("data/zones.csv", header=0)|> Tables.matrix;

model = Model(() -> Gurobi.Optimizer(GRB_ENV))

@variable(model, x[1:20], Bin)
@variable(model, y[1:20, 1:500] >= 0)
@variable(model, z[1:50, 1:20, 1:500] >= 0)
@variable(model, e[1:50, 1:500] >= 0)

@objective(model, Min,
sum(facilities[j, 1] * x[j] for j in 1:20) + # Fixed facility costs
    1/500 * sum(sum(production[j, s] * y[j, s] for j in 1:20) + # Production costs
    sum(sum(transportation[i, j] * z[i, j, s] for j in 1:20) for i in 1:50) + #
    sum(zones[i, 1] * e[i, s] for i in 1:50) # Unmet demand costs
    for s in 1:500))
# Facility production constraint
@constraint(model, [j=1:20, s=1:500], y[j, s] <= facilities[j, 2] * x[j])
# Demand fulfillment constraint
@constraint(model, [i=1:50, s=1:500], sum(z[i, j, s] for j in 1:20) + e[i, s] == demand[i, s])
# Transportation constraint
@constraint(model, [j=1:20, s=1:500], sum(z[i, j, s] for i in 1:50) <= y[j, s])
optimize!(model)
println("Optimal objective value: ", objective_value(model))

LoadError: MethodError: no method matching *(::String31, ::VariableRef)

[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
[0m  *([91m::ChainRulesCore.NotImplemented[39m, ::Any)
[0m[90m   @[39m [32mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/6Pucz/src/[39m[90m[4mtangent_arithmetic.jl:37[24m[39m
[0m  *(::Any, [91m::ChainRulesCore.NotImplemented[39m)
[0m[90m   @[39m [32mChainRulesCore[39m [90m~/.julia/packages/ChainRulesCore/6Pucz/src/[39m[90m[4mtangent_arithmetic.jl:38[24m[39m
[0m  ...


In [120]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

@variable(model, x[1:num_facilities], Bin)
@variable(model, b[1:num_customers, 1:num_scenarios] >= 0) #unmet demand e
@variable(model, c[1:num_customers, 1:num_facilities, 1:num_scenarios] >= 0) #quantity shipped z
@variable(model, a[1:num_facilities, 1:num_scenarios] >= 0) # y

@objective(model, Min,
    sum(c_j[j] * x[j] for j in 1:20) + # Fixed facility costs
    1/500 * sum(
    sum(q_js[j, s] * a[j, s] for j in 1:20) + # Production costs
    sum(sum(t_ij[i, j] * c[i, j, s] for j in 1:20) for i in 1:50) + #
    sum(u_i[i] * b[i, s] for i in 1:50) # Unmet demand costs
    for s in 1:500))

# @objective(model, Min, sum(c_j[j] * x[j] for j in 1:num_facilities + 1/num_scenarios * sum(
#     sum(q_js[j, s] * a[j, s] for j in 1:num_facilities)
#     + sum(sum(t_ij[i, j] * c[i, j, s] for j in 1:num_facilities) for i in 1:num_customers) +
#     sum(u_i[i] * b[i, s] for i in 1:num_customers) for s in 1:num_scenarios)))

@constraint(model, [j in 1:num_facilities, s in 1:num_scenarios], a[j, s] <= C_j[j] * x[j])
@constraint(model, [i in 1:num_customers, s in 1:num_scenarios], sum(c[i, j, s] for j in 1:num_facilities) + b[i, s] == D_is[i, s])
@constraint(model, [j in 1:num_facilities, s in 1:num_scenarios], sum(c[i, j, s] for i in 1:num_customers) <= a[j, s])

optimize!(model)
println(objective_value(model))

1250.1861721981154


In [100]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

# First-stage decision variable: whether to build facility j
@variable(model, x[1:num_facilities], Bin)

# Second-stage decision variables: production and unmet demand
@variable(model, a[1:num_facilities, 1:num_scenarios] >= 0)  # production at facility j in scenario s
@variable(model, b[1:num_customers, 1:num_scenarios] >= 0)   # unmet demand for customer i in scenario s
@variable(model, c[1:num_customers, 1:num_facilities, 1:num_scenarios] >= 0)  # quantity shipped

# Objective function: minimize total costs
@objective(model, Min,
    sum(c_j[j] * x[j] for j in 1:num_facilities) +  # construction cost
    sum(p[s] * q_js[j, s] * a[j, s] for j in 1:num_facilities, s in 1:num_scenarios) +  # production cost
    sum(p[s] * t_ij[i, j] * c[i, j, s] for i in 1:num_customers, j in 1:num_facilities, s in 1:num_scenarios) +  # transportation cost
    sum(p[s] * u_i[i] * b[i, s] for i in 1:num_customers, s in 1:num_scenarios)  # unmet demand cost
)

# @objective(model, Min,
#     sum(c_j[j] * x[j] for j in 1:num_facilities) +  # construction cost
#     1/num_scenarios * sum((sum(q_js[j, s] * a[j, s] for j in 1:num_facilities) + sum(t_ij[i, j] * c[i, j, s] for i in 1:num_customers, j in 1:num_facilities)+ sum(u_i[i] * b[i, s] for i in 1:num_customers)) for s in 1:num_scenarios )
#     #  +  # production cost
#     #  +  # transportation cost
#     #   # unmet demand cost
# )

# Capacity constraint: each facility's production cannot exceed its capacity if built
@constraint(model, [j in 1:num_facilities, s in 1:num_scenarios], a[j, s] <= C_j[j] * x[j])

# Demand satisfaction: each customer's demand must be met by the sum of shipments and unmet demand
@constraint(model, [i in 1:num_customers, s in 1:num_scenarios], sum(c[i, j, s] for j in 1:num_facilities) + b[i, s] == D_is[i, s])

# Production allocation: quantity shipped from facility to customers cannot exceed production
@constraint(model, [j in 1:num_facilities, s in 1:num_scenarios], sum(c[i, j, s] for i in 1:num_customers) <= a[j, s])

optimize!(model)


adaptive_cost = objective_value(model)
println("Optimal Objective Value: ", adaptive_cost)
# println("Facility construction decisions: ", value.(x))
# println("Production levels per facility per scenario: ", value.(a))
# println("Unmet demand per customer per scenario: ", value.(b))

Optimal Objective Value: 1250.186172198513


### Compute Benchmarks

#### Deterministic

In [103]:
deterministic = Model(() -> Gurobi.Optimizer(GRB_ENV))

@variable(deterministic, x[1:num_facilities], Bin)

# Second-stage decision variables: production and unmet demand
@variable(deterministic, a[1:num_facilities] >= 0)  # production at facility j in scenario s
@variable(deterministic, b[1:num_customers] >= 0)   # unmet demand for customer i in scenario s
@variable(deterministic, c[1:num_customers, j=1:num_facilities] >= 0)  # quantity shipped


# avg_production_cost = [mean(q_js[j,:]) for j in 1:num_facilities]
avg_production_cost = mean(q_js, dims=2)
avg_demand = mean(D_is, dims=2)

# Capacity constraint: each facility's production cannot exceed its capacity if built
@constraint(deterministic, [j in 1:num_facilities], a[j] <= C_j[j] * x[j])

# Demand satisfaction: each customer's demand must be met by the sum of shipments and unmet demand
@constraint(deterministic, [i in 1:num_customers], sum(c[i, j] for j in 1:num_facilities) + b[i] >= avg_demand[i])

# Production allocation: quantity shipped from facility to customers cannot exceed production
@constraint(deterministic, [j in 1:num_facilities], sum(c[i, j] for i in 1:num_customers) <= a[j])



@objective(deterministic, Min,
    sum(c_j[j] * x[j] for j in 1:num_facilities) +  # construction cost
    sum(avg_production_cost[j] * a[j] for j in 1:num_facilities) +  # production cost
    sum(t_ij[i, j] * c[i, j] for i in 1:num_customers, j in 1:num_facilities) +  # transportation cost
    sum(u_i[i] * b[i] for i in 1:num_customers)  # unmet demand cost
)

optimize!(deterministic)
facility_EV = value.(x)
println("Total number of facilities constructed is ", sum(facility_EV))
println("The cost is ", objective_value(deterministic))

Total number of facilities constructed is 9.0
The cost is 749.0994844684311


In [104]:
cost_scenario = zeros(num_scenarios);
transport_EV = zeros(num_customers, num_facilities, num_scenarios);
unmet_EV = zeros(num_customers, num_scenarios);
    
for s in 1:num_scenarios
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    @variable(model, a[1:num_facilities] >= 0)  # production at facility j in scenario s
    @variable(model, b[1:num_customers] >= 0) #z
    @variable(model, c[1:num_customers, 1:num_facilities] >= 0) #y
    
    # Capacity constraint: each facility's production cannot exceed its capacity if built
    @constraint(model, [j in 1:num_facilities], a[j] <= C_j[j] * facility_EV[j])

    # Demand satisfaction: each customer's demand must be met by the sum of shipments and unmet demand
    @constraint(model, [i in 1:num_customers], sum(c[i, j] for j in 1:num_facilities) + b[i] >= D_is[i, s])

    # Production allocation: quantity shipped from facility to customers cannot exceed production
    @constraint(model, [j in 1:num_facilities], sum(c[i, j] for i in 1:num_customers) <= a[j])
        
    @objective(model, Min,
                sum(c_j[j] * facility_EV[j] for j in 1:num_facilities) +                   # construction cost
                sum(t_ij[i,j] * c[i,j] for j in 1:num_facilities, i in 1:num_customers) +  # transporation cost
                sum(u_i[i] * b[i] for i in 1:num_customers)) +                          # unmet demand cost
                sum(a[j] * facility_EV[j] for j=1:num_facilities)                  # production cost

    optimize!(model)

    cost_scenario[s] = objective_value(model);
    transport_EV[:,:,s] = value.(c);
    unmet_EV[:,s] = value.(b);
end
    
exp_cost_EV = sum(p[s] * cost_scenario[s] for s in 1:num_scenarios)
println("The deterministic cost is ", exp_cost_EV)

The deterministic cost is 1371.1774064649157


#### Stochastic

In [82]:
cost_scenario = zeros(num_scenarios);
facility_WS = zeros(num_facilities, num_scenarios);
transport_WS = zeros(num_customers, num_facilities, num_scenarios);
unmet_WS = zeros(num_customers, num_scenarios);
    
for s in 1:num_scenarios
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    @variable(model, x[1:num_facilities], Bin)
    @variable(model, a[1:num_facilities] >= 0)  # production at facility j in scenario s
    @variable(model, b[1:num_customers] >= 0)   # unmet demand for customer i in scenario s
    @variable(model, c[1:num_customers, 1:num_facilities] >= 0)  # quantity shipped

    # Capacity constraint: each facility's production cannot exceed its capacity if built
    @constraint(model, [j in 1:num_facilities], a[j] <= C_j[j] * x[j])

    # Demand satisfaction: each customer's demand must be met by the sum of shipments and unmet demand
    @constraint(model, [i in 1:num_customers], sum(c[i, j] for j in 1:num_facilities) + b[i] >= D_is[i, s])

    # Production allocation: quantity shipped from facility to customers cannot exceed production
    @constraint(model, [j in 1:num_facilities], sum(c[i, j] for i in 1:num_customers) <= a[j])

    @objective(model, Min,
                sum(c_j[j] * x[j] for j in 1:num_facilities) +
                sum(t_ij[i,j] * c[i,j] for j in 1:num_facilities, i in 1:num_customers) +
                sum(u_i[i] * b[i] for i in 1:num_customers)) +
                sum(q_js[j,s] * a[j] for j in 1:num_facilities)
    
    optimize!(model)

    cost_scenario[s] = objective_value(model);
    facility_WS[:,s] = value.(x);
    transport_WS[:,:,s] = value.(c);
    unmet_WS[:,s] = value.(b);
end
    
exp_cost_WS = sum(p[s] * cost_scenario[s] for s in 1:num_scenarios);
println("The wait and see cost is ", exp_cost_WS)

The wait and see cost is 650.432524313653


In [91]:
VSS = (exp_cost_EV - adaptive_cost)/exp_cost_EV
EVPI = (adaptive_cost - exp_cost_WS)/adaptive_cost

println("VSS: ", VSS)
println("EVPI: ", EVPI)

VSS: 0.08823893578293146
EVPI: 0.47973146782459775
